# Predicción de la evolución de la diabetes en pacientes usando MLP



Se desea construir um modelo de regresión no lineal (redes neuronales artificiales) que permita pronósticar el progreso de la diabetes con un horizonte de doce meses con base en variables físicas y pruebas de laboratorio. Véase https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

En este problema se tiene una base de datos de diez variables base (edad, sexo, índice de masa corporal, presión arterial, y seis variables medidas en sangre) para 442 pacientes, y un índice que mide el progreso de la diabetes un año después de la prueba. La columna Y es la variable explicada.

In [1]:

# La muestra se encuentra dividida en tres partes:
#
#   * X_train, y_true_train: es la muestra para estimar los parametros optimos
#
#   * X_test, y_true_test: es la muestra para seleccionar la mejor configuración
#
#   * X_val, y_true_val: es la muestra para probar el modelo en productivo
#
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pytest
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

df = pd.read_csv("https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/diabetes.csv")

In [2]:
#
# Use la muestra (X_train, y_true_train) para la estimación
# de los pesos óptimos de la red neuronal.
#
# Seleccione el modelo óptimo como aquel que minimice el error
# cuadrático medio para la muestra (X_test, y_true_test).
#
# Considere únicamente modelos desde una (1) hasta (5) 
# neuronas en la capa oculta. Considere únicamente las
# siguientes semillas para inicializar la red neuronal
# 1000, 1001, 1002, 1003, 1004, 1005.
#
# Compute el error cuadrático medio para la muestra
# (X_val, y_true_val). Esta muestra representa la operación
# del modelo en productio
# 
# Rta/
# True
#
scaler = MinMaxScaler(feature_range=(0,0.5)) #Escalamos los datos del DF para que sean estandares
df_norm = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
X = df_norm.iloc[:,:-1]
y = df_norm['target']

#Particion de los datos
X_train, X_test, y_true_train, y_true_test = train_test_split(X, y, test_size=0.3,random_state=177) #Entrenamos los datos
X_val, X_test, y_true_val, y_true_test = train_test_split(X_test,y_true_test, test_size=0.5,random_state=177) #Entrenamos los datos

param_grid2 = {#Aplicamos GrindSearch
                "hidden_layer_sizes": [(1,),(2,),(3,),(4,),(5,)],
                "random_state": [1000,1001,1002,1004,1005],
    }

#Creamos el modelo
m2 = MLPRegressor(activation="identity",
    learning_rate_init=0.001,
    solver='adam',
    max_iter=100) 

# Creamos el GridSearc
grid_search = GridSearchCV(estimator = m2, param_grid=param_grid2, cv=5) 
# Entrenamos con el Gridsearch
grid_search.fit(X_train, y_true_train)

# Miramos el mejor estimador
#print('Mejor Estimador : ', grid_search.best_estimator_)
#print('Mejores Parametros : ', grid_search.best_params_)
#print('Mejor score : ',grid_search.best_score_)

# Hacemos la prediccion
y_pred = grid_search.predict(X_val)
mse_val = mean_squared_error(y_true_val,y_pred)

# ---->>> Evaluación ---->>>
pytest.approx(mse_val, 0.0001) == 0.009535





True